In [1]:
import duckdb
import glob
import logging
import os
import cyvcf2

In [2]:
logger = logging.getLogger(__name__)

In [3]:
def patientLister(patientPath,db=None):
            vcfspath = glob.glob(patientPath+'**/*.vcf',
                                recursive=True)
            try:
                vcfsnames = [cyvcf2.Reader(x).samples[0] for x in vcfspath]
            except Exception as e:
                logger.info(
                    "No Sample name in one of the vcfs files. Using File Names Instead"
                )
                logger.debug(str(e))
                vcfsnames = [
                    x.rsplit("/", maxsplit=1)[-1].strip(".final.vcf") for x in vcfspath
                ]

            if db is not None:
                addpatnames = [
                    x
                    for x in vcfsnames
                    if (
                        x not in db.columns
                        and x + "_MODApy" not in db.columns
                        and x.replace("_MODApy", "") not in db.columns
                    )
                ]
                if len(addpatnames) >= 1:
                    logger.info("Adding Patients: {}".format([x for x in addpatnames]))
                else:
                    logger.error("No Patients to Add")
                    exit(1)
                patientslist = [x for x in vcfspath for y in addpatnames if y in x]
            else:
                patientslist = vcfspath
            vcfsnames = [x.split('_MODApy')[0] for x in vcfsnames]
            patientslist = [x.replace('.vcf','.parquet') for x in patientslist]
            patientslist = tuple(zip(vcfsnames,patientslist))
            return patientslist

In [4]:
patient_list = patientLister("../../data/Patients/test_patients/",db=None)

In [5]:
patient_list

(('BM23-48263',
  '../../data/Patients/test_patients/BM23-48263_MODApy/BM23-48263_MODApy.final.parquet'),
 ('33745', '../../data/Patients/test_patients/33745/33745.final.parquet'),
 ('32238',
  '../../data/Patients/test_patients/32238/32238.Filtered.Variants.parquet'),
 ('34424', '../../data/Patients/test_patients/34424_A/34424_A.final.parquet'),
 ('34352', '../../data/Patients/test_patients/34352/34352.final.parquet'),
 ('34424', '../../data/Patients/test_patients/34424/34424.final.parquet'),
 ('34082', '../../data/Patients/test_patients/34082/34082.final.parquet'),
 ('31973', '../../data/Patients/test_patients/31973/31973.final.parquet'),
 ('33325', '../../data/Patients/test_patients/33325/33325.final.parquet'),
 ('32186',
  '../../data/Patients/test_patients/32186/32186.Filtered.Variants.parquet'))

In [7]:
conn = duckdb.connect(database='./variants.db', read_only=False)

In [8]:
total = len(patient_list)
counter = 0
table_name = 'exome_patients'
for patient in patient_list:
    try:
        counter += 1
        print(f'Processing file {counter} of {total}')
        print(f'Adding Patient {patient[0]} to {table_name}')
        patient_name = patient[0].replace('-','_').lower()
        create_query = f"""CREATE TABLE IF NOT EXISTS exome_patients AS (
        SELECT *, '{patient_name}' AS sample
        FROM '{patient[1]}'
        );"""    
        query = f"""INSERT INTO {table_name} BY NAME (SELECT *, '{patient_name}' as sample FROM '{patient[1]}' UNION SELECT * FROM {table_name} LIMIT 1);"""
        conn.execute(create_query)
        conn.execute(query)
        print(f'Added Patient {patient[0]} to {table_name}')
    except Exception as e:
        print(f'Error adding patient {patient[0]} to {table_name}')
        print(str(e))
        continue


Processing file 1 of 10
Adding Patient BM23-48263 to exome_patients


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Added Patient BM23-48263 to exome_patients
Processing file 2 of 10
Adding Patient 33745 to exome_patients
Error adding patient 33745 to exome_patients
Binder Error: Set operations can only apply to expressions with the same number of result columns
Processing file 3 of 10
Adding Patient 32238 to exome_patients
Error adding patient 32238 to exome_patients
Binder Error: Set operations can only apply to expressions with the same number of result columns
Processing file 4 of 10
Adding Patient 34424 to exome_patients
Error adding patient 34424 to exome_patients
Binder Error: Set operations can only apply to expressions with the same number of result columns
Processing file 5 of 10
Adding Patient 34352 to exome_patients
Error adding patient 34352 to exome_patients
Binder Error: Set operations can only apply to expressions with the same number of result columns
Processing file 6 of 10
Adding Patient 34424 to exome_patients
Error adding patient 34424 to exome_patients
Binder Error: Set operati

In [9]:
conn.execute("SHOW TABLES").fetchall()

[('exome_patients',)]

In [10]:
df = conn.execute("SELECT * FROM exome_patients LIMIT 10").fetchdf()

In [11]:
df

,CHROM,POS,REF,QUAL,FILTER,AN,BASEQRANKSUM,CLIPPINGRANKSUM,DP,FS,...,RANK,HGVS.C,HGVS.P,CDNA.POS / CDNA.LENGTH,CDS.POS / CDS.LENGTH,AA.POS / AA.LENGTH,DISTANCE,ERRORS / WARNINGS / INFO,AMINOCHANGE,sample
0,chr1,14653,C,48.77,.,2.0,0.217,-0.65,15.0,0.0,...,.,n.*244C>T,.,.,.,.,244,.,.,bm23_48263
1,chr1,14653,C,48.77,.,2.0,0.217,-0.65,15.0,0.0,...,.,n.*2716G>A,.,.,.,.,2716,.,.,bm23_48263
2,chr1,14653,C,48.77,.,2.0,0.217,-0.65,15.0,0.0,...,.,n.*2716G>A,.,.,.,.,2716,.,.,bm23_48263
3,chr1,14653,C,48.77,.,2.0,0.217,-0.65,15.0,0.0,...,.,n.*2716G>A,.,.,.,.,2716,.,.,bm23_48263
4,chr1,14653,C,48.77,.,2.0,0.217,-0.65,15.0,0.0,...,.,n.*2716G>A,.,.,.,.,2716,.,.,bm23_48263
5,chr1,14653,C,48.77,.,2.0,0.217,-0.65,15.0,0.0,...,11/11,n.1478G>A,.,.,.,.,.,.,.,bm23_48263
6,chr1,14671,G,202.770004,.,2.0,-1.555,0.4,18.0,0.0,...,.,n.*262G>C,.,.,.,.,262,.,.,bm23_48263
7,chr1,14671,G,202.770004,.,2.0,-1.555,0.4,18.0,0.0,...,.,n.*2698C>G,.,.,.,.,2698,.,.,bm23_48263
8,chr1,14671,G,202.770004,.,2.0,-1.555,0.4,18.0,0.0,...,.,n.*2698C>G,.,.,.,.,2698,.,.,bm23_48263
9,chr1,14671,G,202.770004,.,2.0,-1.555,0.4,18.0,0.0,...,.,n.*2698C>G,.,.,.,.,2698,.,.,bm23_48263


In [12]:
conn.close()

In [14]:
duckdb.__version__

'0.10.0'

In [8]:
schema = conn.execute(f'SELECT * FROM "{patient_list[0][1]}" LIMIT 1').fetchdf()

In [9]:
schema.dtypes

CHROM                       object
POS                          int64
REF                         object
QUAL                        object
FILTER                      object
                             ...  
CDS.POS / CDS.LENGTH        object
AA.POS / AA.LENGTH          object
DISTANCE                    object
ERRORS / WARNINGS / INFO    object
AMINOCHANGE                 object
Length: 270, dtype: object

In [17]:
col_list = conn.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'exome_patients'").fetchdf().column_name.tolist()